In [1]:
#Importing Data From Kaggle
import os
os.environ['KAGGLE_USERNAME']="oa341791"
os.environ['KAGGLE_KEY']="e1d6525f3a295531cdd1ed595b5f8408"
!kaggle competitions download -c nn23-sports-image-classification

nn23-sports-image-classification.zip: Skipping, found more recently modified local copy (use --force to force download)


In [2]:
#Extract Data
from zipfile import ZipFile
file_name="/content/nn23-sports-image-classification.zip"
with ZipFile(file_name,'r')as zip:
  zip.extractall()
  print("done")

done


In [3]:
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow

In [4]:
CATEGORIES = ['Basketball','Football','Rowing','Swimming','Tennis','Yoga']
trainingData = []
path = '/content/Train'
for img in os.listdir(path):
    try:
      image = cv2.imread(os.path.join(path,img),cv2.IMREAD_UNCHANGED)
      if img.split('_')[0]=='Basketball' :
        imageLabel = 0
      elif img.split('_')[0]=='Football':
        imageLabel = 1
      elif img.split('_')[0]=='Rowing':
        imageLabel = 2
      elif img.split('_')[0]=='Swimming':
        imageLabel = 3
      elif img.split('_')[0]=='Tennis':
        imageLabel = 4
      elif img.split('_')[0]=='Yoga':
        imageLabel = 5
      imageResized = cv2.resize(image,(224,224))
      trainingData.append([imageResized,imageLabel])
    except Exception as e: 
      pass

In [5]:
X = []
Y = []
for features,labels in trainingData:
  X.append(features)
  Y.append(labels)
X = np.array(X)
Y = np.array(Y)
X = X/255

In [6]:
from sklearn.model_selection import train_test_split
X_Train,X_Valid,Y_Train,Y_Valid=train_test_split(X,Y,test_size=0.2,shuffle=True)

In [7]:
######################################################MobileNetV2################################################################################
import tensorflow.compat.v2 as tf

from keras import backend
from keras.applications import imagenet_utils
from keras.engine import training
from keras.layers import VersionAwareLayers
from keras.utils import data_utils
from keras.utils import layer_utils
# isort: off
from tensorflow.python.platform import tf_logging as logging
from tensorflow.python.util.tf_export import keras_export

BASE_WEIGHT_PATH = (
    "https://storage.googleapis.com/tensorflow/keras-applications/mobilenet_v2/"
)
layers = None
def MobileNetV2(
    input_shape=None,
    alpha=1.0,
    include_top=True,
    weights="imagenet",
    input_tensor=None,
    pooling=None,
    classes=1000,
    classifier_activation="softmax",
    **kwargs,
):
    global layers
    if "layers" in kwargs:
        layers = kwargs.pop("layers")
    else:
        layers = VersionAwareLayers()
    # Determine proper input shape and default size.
    # If both input_shape and input_tensor are used, they should match
    # If input_shape is None and no input_tensor
    if input_shape is None:
        default_size = 224
    # If input_shape is not None, assume default size.
    else:
        if backend.image_data_format() == "channels_first":
            rows = input_shape[1]
            cols = input_shape[2]
        else:
            rows = input_shape[0]
            cols = input_shape[1]

        if rows == cols and rows in [96, 128, 160, 192, 224]:
            default_size = rows
        else:
            default_size = 224

    input_shape = imagenet_utils.obtain_input_shape(
        input_shape,
        default_size=default_size,
        min_size=32,
        data_format=backend.image_data_format(),
        require_flatten=include_top,
        weights=weights,
    )

    if backend.image_data_format() == "channels_last":
        row_axis, col_axis = (0, 1)
    else:
        row_axis, col_axis = (1, 2)
    rows = input_shape[row_axis]
    cols = input_shape[col_axis]
    if input_tensor is None:
        img_input = layers.Input(shape=input_shape)
    channel_axis = 1 if backend.image_data_format() == "channels_first" else -1
    first_block_filters = _make_divisible(32 * alpha, 8)
    x = layers.Conv2D(first_block_filters,kernel_size=3,strides=(2, 2),padding="same",use_bias=False,name="Conv1",)(img_input)
    x = layers.BatchNormalization(axis=channel_axis, epsilon=1e-3, momentum=0.999, name="bn_Conv1")(x)
    x = layers.ReLU(6.0, name="Conv1_relu")(x)
    x = _inverted_res_block(x, filters=16, alpha=alpha, stride=1, expansion=1, block_id=0)
    x = _inverted_res_block(x, filters=24, alpha=alpha, stride=2, expansion=6, block_id=1)
    x = _inverted_res_block(x, filters=24, alpha=alpha, stride=1, expansion=6, block_id=2)
    x = _inverted_res_block(x, filters=32, alpha=alpha, stride=2, expansion=6, block_id=3)
    x = _inverted_res_block(x, filters=32, alpha=alpha, stride=1, expansion=6, block_id=4)
    x = _inverted_res_block(x, filters=32, alpha=alpha, stride=1, expansion=6, block_id=5)
    x = _inverted_res_block(x, filters=64, alpha=alpha, stride=2, expansion=6, block_id=6)
    x = _inverted_res_block(x, filters=64, alpha=alpha, stride=1, expansion=6, block_id=7)
    x = _inverted_res_block(x, filters=64, alpha=alpha, stride=1, expansion=6, block_id=8)
    x = _inverted_res_block(x, filters=64, alpha=alpha, stride=1, expansion=6, block_id=9)
    x = _inverted_res_block(x, filters=96, alpha=alpha, stride=1, expansion=6, block_id=10)
    x = _inverted_res_block(x, filters=96, alpha=alpha, stride=1, expansion=6, block_id=11)
    x = _inverted_res_block(x, filters=96, alpha=alpha, stride=1, expansion=6, block_id=12)
    x = _inverted_res_block(x, filters=160, alpha=alpha, stride=2, expansion=6, block_id=13)
    x = _inverted_res_block(x, filters=160, alpha=alpha, stride=1, expansion=6, block_id=14)
    x = _inverted_res_block(x, filters=160, alpha=alpha, stride=1, expansion=6, block_id=15)
    x = _inverted_res_block(x, filters=320, alpha=alpha, stride=1, expansion=6, block_id=16)
    if alpha > 1.0:
        last_block_filters = _make_divisible(1280 * alpha, 8)
    else:
        last_block_filters = 1280
    x = layers.Conv2D(last_block_filters, kernel_size=1, use_bias=False, name="Conv_1")(x)
    x = layers.BatchNormalization(axis=channel_axis, epsilon=1e-3, momentum=0.999, name="Conv_1_bn")(x)
    x = layers.ReLU(6.0, name="out_relu")(x)
    if pooling == "avg":
      x = layers.GlobalAveragePooling2D()(x)
    elif pooling == "max":
      x = layers.GlobalMaxPooling2D()(x)
    inputs = img_input
    # Create model.
    model = training.Model(inputs, x, name=f"mobilenetv2_{alpha:0.2f}_{rows}")
    # Load weights.
    if weights == "imagenet":
      model_name = ("mobilenet_v2_weights_tf_dim_ordering_tf_kernels_"+ str(float(alpha))+ "_" + str(rows)+ "_no_top"+ ".h5")
      weight_path = BASE_WEIGHT_PATH + model_name
      weights_path = data_utils.get_file(model_name, weight_path, cache_subdir="models")
      model.load_weights(weights_path)
    return model
def _inverted_res_block(inputs, expansion, stride, alpha, filters, block_id):
    """Inverted ResNet block."""
    channel_axis = 1 if backend.image_data_format() == "channels_first" else -1
    in_channels = backend.int_shape(inputs)[channel_axis]
    pointwise_conv_filters = int(filters * alpha)
    # Ensure the number of filters on the last 1x1 convolution is divisible by 8.
    pointwise_filters = _make_divisible(pointwise_conv_filters, 8)
    x = inputs
    prefix = f"block_{block_id}_"
    if block_id:
        # Expand with a pointwise 1x1 convolution.
        x = layers.Conv2D(expansion * in_channels,kernel_size=1,padding="same",use_bias=False,activation=None,name=prefix + "expand",)(x)
        x = layers.BatchNormalization(axis=channel_axis,epsilon=1e-3,momentum=0.999,name=prefix + "expand_BN",)(x)
        x = layers.ReLU(6.0, name=prefix + "expand_relu")(x)
    else:
        prefix = "expanded_conv_"
    # Depthwise 3x3 convolution.
    if stride == 2:x = layers.ZeroPadding2D(padding=imagenet_utils.correct_pad(x, 3), name=prefix + "pad")(x)
    x = layers.DepthwiseConv2D(kernel_size=3,strides=stride,activation=None,use_bias=False,padding="same" if stride == 1 else "valid",name=prefix + "depthwise",)(x)
    x = layers.BatchNormalization(axis=channel_axis,epsilon=1e-3,momentum=0.999,name=prefix + "depthwise_BN",)(x)
    x = layers.ReLU(6.0, name=prefix + "depthwise_relu")(x)
    # Project with a pointwise 1x1 convolution.
    x = layers.Conv2D(pointwise_filters,kernel_size=1,padding="same",use_bias=False,activation=None,name=prefix + "project",)(x)
    x = layers.BatchNormalization(axis=channel_axis,epsilon=1e-3,momentum=0.999,name=prefix + "project_BN",)(x)
    if in_channels == pointwise_filters and stride == 1:
        return layers.Add(name=prefix + "add")([inputs, x])
    return x
def _make_divisible(v, divisor, min_value=None):
    if min_value is None:
        min_value = divisor
    new_v = max(min_value, int(v + divisor / 2) // divisor * divisor)
    # Make sure that round down does not go down by more than 10%.
    if new_v < 0.9 * v:
        new_v += divisor
    return new_v

In [8]:
mob2=MobileNetV2(input_shape=(224,224,3),weights='imagenet',include_top=False)

In [9]:
for layer in mob2.layers:
  layer.trainable=False
mob2.summary()

Model: "mobilenetv2_1.00_224"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 112, 112, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 112, 112, 32  128         ['Conv1[0][0]']                  
                                )                                              

In [10]:
from keras.layers import Flatten,Dense
x=Flatten()(mob2.output)
prediction8=Dense(6,activation='softmax')(x)

In [11]:
from keras.models import Model
model2=Model(inputs=mob2.input,outputs=prediction8)
model2.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [12]:
from keras.callbacks import EarlyStopping
model2.fit(X_Train,Y_Train,epochs=50)

Epoch 1/50
42/42 [==============================] - 6s 44ms/step - loss: 2.7921 - accuracy: 0.7932
Epoch 2/50
42/42 [==============================] - 2s 44ms/step - loss: 0.1515 - accuracy: 0.9777
Epoch 3/50
42/42 [==============================] - 2s 44ms/step - loss: 0.1215 - accuracy: 0.9851
Epoch 4/50
42/42 [==============================] - 2s 44ms/step - loss: 0.0137 - accuracy: 0.9948
Epoch 5/50
42/42 [==============================] - 2s 44ms/step - loss: 0.0053 - accuracy: 0.9985
Epoch 6/50
42/42 [==============================] - 2s 44ms/step - loss: 0.0079 - accuracy: 0.9985
Epoch 7/50
42/42 [==============================] - 2s 45ms/step - loss: 0.0058 - accuracy: 0.9985
Epoch 8/50
42/42 [==============================] - 2s 44ms/step - loss: 0.0123 - accuracy: 0.9970
Epoch 9/50
42/42 [==============================] - 2s 44ms/step - loss: 0.0090 - accuracy: 0.9985
Epoch 10/50
42/42 [==============================] - 2s 45ms/step - loss: 0.0067 - accuracy: 0.9970
Epoch 11/

In [13]:
# Importing Libraries Needed
import tensorflow.compat.v2 as tf
from keras import backend
from keras.applications import imagenet_utils
from keras.engine import training
from keras.layers import VersionAwareLayers
from keras.utils import data_utils
from keras.utils import layer_utils
from tensorflow.python.util.tf_export import keras_export
layers = VersionAwareLayers()
# Needed Weights for Transfer Learning
TF_WEIGHTS_PATH_NO_TOP = (
    "https://storage.googleapis.com/tensorflow/keras-applications/"
    "xception/xception_weights_tf_dim_ordering_tf_kernels_notop.h5"
)
def Xception(
    # Determining The Required Arguments and Their Default Values
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    input_shape=None,
    pooling=None,
    classes=1000,
    classifier_activation="softmax",
):
    # Determine The Input Shape and its default
    input_shape = imagenet_utils.obtain_input_shape(
        input_shape,
        default_size=299,
        min_size=71,
        data_format=backend.image_data_format(),
        require_flatten=include_top,
        weights=weights,
    )
    #Entry Flow - changing Dimensionality through Strides
    # As we will not apply Input Tesnor then The First layer is Input Shape
    if input_tensor is None:
        img_input = layers.Input(shape=input_shape)
    channel_axis = 1 if backend.image_data_format() == "channels_first" else -1
    # Using Separable Convolutions to perform depth-wise convolutions
    x = layers.Conv2D(32, (3, 3), strides=(2, 2), use_bias=False, name="block1_conv1")(img_input)
    x = layers.BatchNormalization(axis=channel_axis, name="block1_conv1_bn")(x)
    x = layers.Activation("relu", name="block1_conv1_act")(x)
    x = layers.Conv2D(64, (3, 3), use_bias=False, name="block1_conv2")(x)
    x = layers.BatchNormalization(axis=channel_axis, name="block1_conv2_bn")(x)
    x = layers.Activation("relu", name="block1_conv2_act")(x)
    residual = layers.Conv2D(128, (1, 1), strides=(2, 2), padding="same", use_bias=False)(x)
    residual = layers.BatchNormalization(axis=channel_axis)(residual)
    x = layers.SeparableConv2D(128, (3, 3), padding="same", use_bias=False, name="block2_sepconv1")(x)
    x = layers.BatchNormalization(axis=channel_axis, name="block2_sepconv1_bn")(x)
    x = layers.Activation("relu", name="block2_sepconv2_act")(x)
    x = layers.SeparableConv2D(128, (3, 3), padding="same", use_bias=False, name="block2_sepconv2")(x)
    x = layers.BatchNormalization(axis=channel_axis, name="block2_sepconv2_bn")(x)
    x = layers.MaxPooling2D((3, 3), strides=(2, 2), padding="same", name="block2_pool")(x)
    x = layers.add([x, residual])
    residual = layers.Conv2D(256, (1, 1), strides=(2, 2), padding="same", use_bias=False)(x)
    residual = layers.BatchNormalization(axis=channel_axis)(residual)
    x = layers.Activation("relu", name="block3_sepconv1_act")(x)
    x = layers.SeparableConv2D(256, (3, 3), padding="same", use_bias=False, name="block3_sepconv1")(x)
    x = layers.BatchNormalization(axis=channel_axis, name="block3_sepconv1_bn")(x)
    x = layers.Activation("relu", name="block3_sepconv2_act")(x)
    x = layers.SeparableConv2D(256, (3, 3), padding="same", use_bias=False, name="block3_sepconv2")(x)
    x = layers.BatchNormalization(axis=channel_axis, name="block3_sepconv2_bn")(x)
    x = layers.MaxPooling2D((3, 3), strides=(2, 2), padding="same", name="block3_pool")(x)
    x = layers.add([x, residual])
    residual = layers.Conv2D(728, (1, 1), strides=(2, 2), padding="same", use_bias=False)(x)
    residual = layers.BatchNormalization(axis=channel_axis)(residual)
    x = layers.Activation("relu", name="block4_sepconv1_act")(x)
    x = layers.SeparableConv2D(728, (3, 3), padding="same", use_bias=False, name="block4_sepconv1")(x)
    x = layers.BatchNormalization(axis=channel_axis, name="block4_sepconv1_bn")(x)
    x = layers.Activation("relu", name="block4_sepconv2_act")(x)
    x = layers.SeparableConv2D(728, (3, 3), padding="same", use_bias=False, name="block4_sepconv2")(x)
    x = layers.BatchNormalization(axis=channel_axis, name="block4_sepconv2_bn")(x)
    x = layers.MaxPooling2D((3, 3), strides=(2, 2), padding="same", name="block4_pool")(x)
    x = layers.add([x, residual])
    # Middle Flow - doesn't change dimensions
    for i in range(8):
        residual = x
        prefix = "block" + str(i + 5)
        x = layers.Activation("relu", name=prefix + "_sepconv1_act")(x)
        x = layers.SeparableConv2D(728,(3, 3),padding="same",use_bias=False,name=prefix + "_sepconv1",)(x)
        x = layers.BatchNormalization(axis=channel_axis, name=prefix + "_sepconv1_bn")(x)
        x = layers.Activation("relu", name=prefix + "_sepconv2_act")(x)
        x = layers.SeparableConv2D(728,(3, 3),padding="same",use_bias=False,name=prefix + "_sepconv2",)(x)
        x = layers.BatchNormalization(axis=channel_axis, name=prefix + "_sepconv2_bn")(x)
        x = layers.Activation("relu", name=prefix + "_sepconv3_act")(x)
        x = layers.SeparableConv2D(728,(3, 3),padding="same",use_bias=False,name=prefix + "_sepconv3",)(x)
        x = layers.BatchNormalization(axis=channel_axis, name=prefix + "_sepconv3_bn")(x)
        x = layers.add([x, residual])
    # Exit Flow has The Ability to change Dimensions through strides 
    residual = layers.Conv2D(1024, (1, 1), strides=(2, 2), padding="same", use_bias=False)(x)
    residual = layers.BatchNormalization(axis=channel_axis)(residual)
    x = layers.Activation("relu", name="block13_sepconv1_act")(x)
    x = layers.SeparableConv2D(728, (3, 3), padding="same", use_bias=False, name="block13_sepconv1")(x)
    x = layers.BatchNormalization(axis=channel_axis, name="block13_sepconv1_bn")(x)
    x = layers.Activation("relu", name="block13_sepconv2_act")(x)
    x = layers.SeparableConv2D(1024, (3, 3), padding="same", use_bias=False, name="block13_sepconv2")(x)
    x = layers.BatchNormalization(axis=channel_axis, name="block13_sepconv2_bn")(x)
    x = layers.MaxPooling2D((3, 3), strides=(2, 2), padding="same", name="block13_pool")(x)
    x = layers.add([x, residual])
    x = layers.SeparableConv2D(1536, (3, 3), padding="same", use_bias=False, name="block14_sepconv1")(x)
    x = layers.BatchNormalization(axis=channel_axis, name="block14_sepconv1_bn")(x)
    x = layers.Activation("relu", name="block14_sepconv1_act")(x)
    x = layers.SeparableConv2D(2048, (3, 3), padding="same", use_bias=False, name="block14_sepconv2")(x)
    x = layers.BatchNormalization(axis=channel_axis, name="block14_sepconv2_bn")(x)
    x = layers.Activation("relu", name="block14_sepconv2_act")(x)
    if pooling == "avg":
      x = layers.GlobalAveragePooling2D()(x)
    elif pooling == "max":
      x = layers.GlobalMaxPooling2D()(x)
    inputs = img_input
    # Creating Model
    model = training.Model(inputs, x, name="xception")
    # Giving It weights of Image Net 
    if weights == "imagenet":
        weights_path = data_utils.get_file("xception_weights_tf_dim_ordering_tf_kernels_notop.h5",TF_WEIGHTS_PATH_NO_TOP,cache_subdir="models",file_hash="b0042744bf5b25fce3cb969f33bebb97",)
        model.load_weights(weights_path)
    return model

In [14]:
xception=Xception(input_shape=(224,224,3),weights = 'imagenet',include_top=False) 
for layer in xception.layers:
  layer.trainable=False

In [15]:
from keras.layers import Flatten,Dense
x=Flatten()(xception.output)
prediction7=Dense(6,activation='softmax')(x)

In [16]:
from keras.models import Model
model3=Model(inputs=xception.input,outputs=prediction7)
model3.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model3.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 block1_conv1 (Conv2D)          (None, 111, 111, 32  864         ['input_2[0][0]']                
                                )                                                                 
                                                                                                  
 block1_conv1_bn (BatchNormaliz  (None, 111, 111, 32  128        ['block1_conv1[0][0]']           
 ation)                         )                                                           

In [17]:
from keras.callbacks import EarlyStopping
model3.fit(X_Train,Y_Train,epochs=25)

Epoch 1/25
42/42 [==============================] - 9s 126ms/step - loss: 1.6823 - accuracy: 0.8244
Epoch 2/25
42/42 [==============================] - 5s 126ms/step - loss: 0.1537 - accuracy: 0.9688
Epoch 3/25
42/42 [==============================] - 5s 126ms/step - loss: 0.0517 - accuracy: 0.9881
Epoch 4/25
42/42 [==============================] - 5s 127ms/step - loss: 0.0435 - accuracy: 0.9859
Epoch 5/25
42/42 [==============================] - 5s 127ms/step - loss: 0.1366 - accuracy: 0.9799
Epoch 6/25
42/42 [==============================] - 5s 128ms/step - loss: 0.0537 - accuracy: 0.9888
Epoch 7/25
42/42 [==============================] - 5s 128ms/step - loss: 0.0128 - accuracy: 0.9978
Epoch 8/25
42/42 [==============================] - 5s 129ms/step - loss: 0.0022 - accuracy: 0.9985
Epoch 9/25
42/42 [==============================] - 5s 129ms/step - loss: 8.8760e-05 - accuracy: 1.0000
Epoch 10/25
42/42 [==============================] - 5s 128ms/step - loss: 3.4536e-04 - accuracy

In [18]:
import tensorflow.compat.v2 as tf
from keras import backend
from keras.applications import imagenet_utils
from keras.engine import training
from keras.layers import VersionAwareLayers
from keras.utils import data_utils
from keras.utils import layer_utils
BASE_WEIGHTS_PATH = (
    "https://storage.googleapis.com/tensorflow/keras-applications/densenet/"
)
DENSENET201_WEIGHT_PATH_NO_TOP = (
    BASE_WEIGHTS_PATH
    + "densenet201_weights_tf_dim_ordering_tf_kernels_notop.h5"
)
layers = VersionAwareLayers()
def dense_block(x, blocks, name):
    for i in range(blocks):
        x = conv_block(x, 32, name=name + "_block" + str(i + 1))
    return x
def transition_block(x, reduction, name):
    bn_axis = 3 if backend.image_data_format() == "channels_last" else 1
    x = layers.BatchNormalization(axis=bn_axis, epsilon=1.001e-5, name=name + "_bn")(x)
    x = layers.Activation("relu", name=name + "_relu")(x)
    x = layers.Conv2D(int(backend.int_shape(x)[bn_axis] * reduction),1,use_bias=False,name=name + "_conv",)(x)
    x = layers.AveragePooling2D(2, strides=2, name=name + "_pool")(x)
    return x
def conv_block(x, growth_rate, name):
    bn_axis = 3 if backend.image_data_format() == "channels_last" else 1
    x1 = layers.BatchNormalization(axis=bn_axis, epsilon=1.001e-5, name=name + "_0_bn")(x)
    x1 = layers.Activation("relu", name=name + "_0_relu")(x1)
    x1 = layers.Conv2D(4 * growth_rate, 1, use_bias=False, name=name + "_1_conv")(x1)
    x1 = layers.BatchNormalization(axis=bn_axis, epsilon=1.001e-5, name=name + "_1_bn")(x1)
    x1 = layers.Activation("relu", name=name + "_1_relu")(x1)
    x1 = layers.Conv2D(growth_rate, 3, padding="same", use_bias=False, name=name + "_2_conv")(x1)
    x = layers.Concatenate(axis=bn_axis, name=name + "_concat")([x, x1])
    return x
def DenseNet(
    blocks,
    include_top=True,
    weights="imagenet",
    input_tensor=None,
    input_shape=None,
    pooling=None,
    classes=1000,
    classifier_activation="softmax",
):
    # Determine proper input shape
    input_shape = imagenet_utils.obtain_input_shape(
        input_shape,
        default_size=224,
        min_size=32,
        data_format=backend.image_data_format(),
        require_flatten=include_top,
        weights=weights,
    )
    if input_tensor is None:
        img_input = layers.Input(shape=input_shape)
    else:
        if not backend.is_keras_tensor(input_tensor):
            img_input = layers.Input(tensor=input_tensor, shape=input_shape)
        else:
            img_input = input_tensor

    bn_axis = 3 if backend.image_data_format() == "channels_last" else 1

    x = layers.ZeroPadding2D(padding=((3, 3), (3, 3)))(img_input)
    x = layers.Conv2D(64, 7, strides=2, use_bias=False, name="conv1/conv")(x)
    x = layers.BatchNormalization(axis=bn_axis, epsilon=1.001e-5, name="conv1/bn")(x)
    x = layers.Activation("relu", name="conv1/relu")(x)
    x = layers.ZeroPadding2D(padding=((1, 1), (1, 1)))(x)
    x = layers.MaxPooling2D(3, strides=2, name="pool1")(x)
    x = dense_block(x, blocks[0], name="conv2")
    x = transition_block(x, 0.5, name="pool2")
    x = dense_block(x, blocks[1], name="conv3")
    x = transition_block(x, 0.5, name="pool3")
    x = dense_block(x, blocks[2], name="conv4")
    x = transition_block(x, 0.5, name="pool4")
    x = dense_block(x, blocks[3], name="conv5")

    x = layers.BatchNormalization(axis=bn_axis, epsilon=1.001e-5, name="bn")(x)
    x = layers.Activation("relu", name="relu")(x)
    if pooling == "avg":
        x = layers.GlobalAveragePooling2D(name="avg_pool")(x)
    elif pooling == "max":
        x = layers.GlobalMaxPooling2D(name="max_pool")(x)
    # Ensure that the model takes into account
    # any potential predecessors of `input_tensor`.
    inputs = img_input
    # Create model.
    if blocks == [6, 12, 48, 32]:
        model = training.Model(inputs, x, name="densenet201")
    # Load weights.
    if weights == "imagenet":

      if blocks == [6, 12, 48, 32]:

        weights_path = data_utils.get_file("densenet201_weights_tf_dim_ordering_tf_kernels_notop.h5",DENSENET201_WEIGHT_PATH_NO_TOP,cache_subdir="models",file_hash="c13680b51ded0fb44dff2d8f86ac8bb1",)
        model.load_weights(weights_path)
    return model
def DenseNet201(
    include_top=True,
    weights="imagenet",
    input_tensor=None,
    input_shape=None,
    pooling=None,
    classes=1000,
    classifier_activation="softmax",
):
    return DenseNet(
        [6, 12, 48, 32],
        include_top,
        weights,
        input_tensor,
        input_shape,
        pooling,
        classes,
        classifier_activation,
    )

In [19]:
dense=DenseNet201(input_shape=(224,224,3),weights = 'imagenet',include_top=False) 
for layer in dense.layers:
  layer.trainable=False

In [20]:
from keras.layers import Flatten,Dense
x=Flatten()(dense.output)
prediction12=Dense(6,activation='softmax')(x)

In [21]:
from keras.models import Model
model4=Model(inputs=dense.input,outputs=prediction12)
model4.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model4.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 zero_padding2d (ZeroPadding2D)  (None, 230, 230, 3)  0          ['input_3[0][0]']                
                                                                                                  
 conv1/conv (Conv2D)            (None, 112, 112, 64  9408        ['zero_padding2d[0][0]']         
                                )                                                                 
                                                                                            

In [22]:
model4.fit(X_Train,Y_Train,epochs=45)

Epoch 1/45
42/42 [==============================] - 21s 146ms/step - loss: 4.8625 - accuracy: 0.7292
Epoch 2/45
42/42 [==============================] - 6s 146ms/step - loss: 0.3492 - accuracy: 0.9650
Epoch 3/45
42/42 [==============================] - 6s 147ms/step - loss: 0.1201 - accuracy: 0.9792
Epoch 4/45
42/42 [==============================] - 6s 146ms/step - loss: 0.0403 - accuracy: 0.9970
Epoch 5/45
42/42 [==============================] - 6s 147ms/step - loss: 7.2430e-04 - accuracy: 1.0000
Epoch 6/45
42/42 [==============================] - 6s 146ms/step - loss: 4.1405e-05 - accuracy: 1.0000
Epoch 7/45
42/42 [==============================] - 6s 147ms/step - loss: 1.7017e-05 - accuracy: 1.0000
Epoch 8/45
42/42 [==============================] - 6s 148ms/step - loss: 1.1536e-05 - accuracy: 1.0000
Epoch 9/45
42/42 [==============================] - 6s 147ms/step - loss: 9.2862e-06 - accuracy: 1.0000
Epoch 10/45
42/42 [==============================] - 6s 152ms/step - loss: 8.04

In [23]:
model2.save('mobilenetv2.hdf5')
model3.save('xception.hdf5')
model4.save('densenet.hdf5')

In [24]:
from keras.models import load_model
model2=load_model("mobilenetv2.hdf5")
model3=load_model('xception.hdf5')
model4=load_model('densenet.hdf5')

In [25]:
models=[model2,model3,model4]
preds=[model.predict(X_Valid)for model in models]
preds=np.array(preds)
summed=np.sum(preds,axis=0)

11/11 [==============================] - 6s 353ms/step


In [26]:
ensemble_prediction=np.argmax(summed,axis=1)
prediction2=model2.predict(X_Valid)
classes2=np.argmax(prediction2,axis=1)
prediction3=model3.predict(X_Valid)
classes3=np.argmax(prediction3,axis=1)
prediction4=model4.predict(X_Valid)
classes4=np.argmax(prediction3,axis=1)

11/11 [==============================] - 1s 138ms/step


In [27]:
from sklearn.metrics import accuracy_score 
accuracy2=accuracy_score(Y_Valid,classes2)
accuracy3=accuracy_score(Y_Valid,classes3)
accuracy4=accuracy_score(Y_Valid,classes4)
ensemble=accuracy_score(Y_Valid,ensemble_prediction)

In [28]:
print("Accuracy 2 : ",accuracy2)
print("Accuracy 3 : ",accuracy3)
print("Accuracy 4 : ",accuracy4)
print("Ensemble Accuracy  : ",ensemble)

Accuracy 2 :  0.9347181008902077
Accuracy 3 :  0.9317507418397626
Accuracy 4 :  0.9317507418397626
Ensemble Accuracy  :  0.9643916913946587
